In [ ]:
from Bio import Entrez
from xml.etree import ElementTree as ET
import re
import requests
Entrez.email = "asamatt@umich.edu"

In [ ]:
handle = Entrez.esearch(db="pubmed", retmax=10, term="Brucella vaccine")
record = Entrez.read(handle)
print(record)

In [ ]:
def convert_pmid_to_pmcid(pmid):
  url = f"https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/?ids={pmid}&format=json"
  response = requests.get(url)
  data = response.json()
  if "records" in data and data["records"]:
    return data["records"][0].get("pmcid", None)
  return None

In [ ]:
print(type(record))
if isinstance(record, dict):
	print(record.keys())
	print(record['IdList'])
	abstracts = []
	for id in record['IdList']:
		pmcid = convert_pmid_to_pmcid(id)
		print(pmcid)
		if pmcid:
			pmcid_handle = Entrez.efetch(db="pmc", id=pmcid, rettype="abstract", retmode="text")
			abstract = pmcid_handle.read()
			print(abstract)
			abstracts.append(abstract)
   

In [ ]:
def get_element_text(elem):
    """Recursively extract all text from an ElementTree element."""
    text_parts = []
    if elem.text:
        text_parts.append(elem.text)
    for child in elem:
        text_parts.append(get_element_text(child))
        if child.tail:
            text_parts.append(child.tail)
    return "".join(text_parts)

In [ ]:
print(type(record))
if isinstance(record, dict):
	print(record.keys())
	print(record['IdList'])
	abstracts = []
	for id in record['IdList']:
		pmid_handle = Entrez.efetch(db="pubmed", id=id, rettype="abstract", retmode="xml")
		xml_data = pmid_handle.read()
		root = ET.fromstring(xml_data)
		for abstract_text in root.findall(".//Abstract"):
			abstracts.append({"pmid": id, "abstract": get_element_text(abstract_text)})

In [ ]:
for abstract in abstracts:
  print(abstract)
  print("\n")

In [ ]:
print(abstracts[0])
clean_abstract = re.search(r"<abstract.*?>.*?</abstract>", abstracts[0], re.DOTALL)

In [ ]:
print(clean_abstract.group(0))